In [1]:
import numpy as np
import scipy.io
import scipy
import numpy.matlib
import scipy.stats
from sklearn.decomposition import PCA
import sys
import scipy.sparse as sps
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
def add_dependencies():
    '''Imports the liblinear libraries to train the MMDT model.'''
    sys.path.append('./src/liblinear-weights-2.21/python/')
add_dependencies()


In [3]:
from liblinearutil import predict, train

In [4]:
def normalizedKernel(K):
    '''Takes a kernel and returns its normalized
    verion to be used further'''
    if K.shape[0] != K.shape[1]:
        return
    DD = np.abs(np.diag(K))
    sqr_DD = np.sqrt(DD).reshape(1,-1)
    sq_DD2 = sqr_DD.T.dot(sqr_DD)
    K = K / sq_DD2
    return K

def formKernel(X1, X2, param):
    '''Constructs a kernel to be used for further tasks'''
    K = np.matmul(X1, X2.T)
    K = normalizedKernel(K)
    return K

In [5]:
def Config(source=None, target=None):
    '''Initiazation of necessary parametrs for compuatation
    
    Input::
        source(string) : source domain index (Default : webcam)
        target(string) : target domain index (Default : dslr)
    Output::
        param : Output dictionary containing parameters'''
    amazon = 0
    webcam = 1
    dslr = 2
    caltech = 3
    
    param = {}
    param['domains'] = [amazon, webcam, dslr, caltech]
    param['domain_names'] = ['amazon', 'webcam', 'dslr', 'caltech']
    
    param['categories'] = ['back_pack', 'bike', 'calculator',
                           'headphones', 'keyboard', 'laptop_computer', 'monitor', 'mouse',
                           'mug', 'projector']
    
    param['DATA_DIR'] = './data/original_data/'
    param['held_out_categories'] = False
    if source is None:
        param['source'] = webcam
    else:
        param['source'] = source
    
    if target is None:
        param['target'] = dslr
    else:
        param['target'] = target
    
    param['num_trials'] = 20
    param['dim'] = 20
    param['C_s'] = 0.05
    param['C_t'] = 1
    param['mmdt_iter'] = 2
    
    if param['source'] == amazon:
        param['num_train_source'] = 20
    else:
        param['num_train_source'] = 8
    
    param['num_train_target'] = 3
    
    param['result_filename'] = './data/DataSplitsOfficeCaltech/SameCategory_{0}-{1}_{2}RandomTrials_10Categories.mat'.format(param['domain_names'][param['source']],
                                                                     param['domain_names'][param['target']],
                                                                     param['num_trials'])
    param['telapsed'] = {}
    return param

In [6]:
def NormData(fts):
    '''Normalizes the data and returns the zscore'''
    sqr_fts = np.sqrt(np.sum(fts**2, 1))
    sqr_fts = np.matrix(sqr_fts).T
    sqr_fts = np.matlib.repmat(sqr_fts, 1, fts.shape[1])
    fts = fts / sqr_fts
    return scipy.stats.zscore(fts)

In [7]:
def LoadOfficePlusCaltechData(foldername):
    '''Loads the Office and Caltech Dataset and returns the Data and corresponding Labels
    
    Input::
        foldername(string) : Path to the dataset
    Output::
        Data(list)   : SURF features of the dataset.
        Labels(list) : Corresponding labels of the images in the dataset.
    '''
    domain_names = ['amazon_SURF_L10.mat', 'webcam_SURF_L10.mat', 'dslr_SURF_L10.mat', 'Caltech10_SURF_L10.mat']
    Data = []
    Labels = []
    for idx, name in enumerate(domain_names):
        fullfilename = foldername + name
        obj = scipy.io.loadmat(fullfilename)
        
        fts = obj['fts']
        labels = obj['labels']
        fts = NormData(fts)
        Data.append(fts)
        Labels.append(labels)
        
    return Data, Labels

In [8]:
param = Config()

In [9]:
[Data, Labels] = LoadOfficePlusCaltechData( param['DATA_DIR'])

In [10]:
n = param['num_trials']
telapsed = np.zeros((n,1))
accuracy = np.zeros((n,1))
pred_labels = np.zeros((n, 1))

In [11]:
def UpdateLabels(labels, all_categories, categories):
    '''Updates labels for all categories'''
    for i in range(0, labels.size):
        oldL = labels[i]
        l = all_categories[oldL]
        newL = l.index(categories)
        labels[i] = newL;
    return labels    

In [12]:
def UpdateLabelValues(labels, param):
    '''Updates labels for the training source, target and testing target dataset.'''
    if 'all_categories' not in param:
        return labels
    all_categories = param['all_categories']
    categories = param['categories']
    labels['train']['source'] = UpdateLabels(labels['train']['source'], all_categories, categories)
    labels['train']['target'] = UpdateLabels(labels['train']['target'], all_categories, categories)
    labels['test']['target'] = UpdateLabels(labels['test']['target'], all_categories, categories)
    return labels

In [ ]:
def AsymmetricFrob_slack_kernel2(KA,KB,C,gamma=None,thresh=None):
    #Frobenius-based transformation learning
    if thresh is None:
        thresh=10e-3;

    if gamma is None:
        gamma = 1e1;

    maxit = 1e6;
    
    [nA,nA] = KA.shape
    [nB,nB] = KB.shape
    S = np.zeros((nA,nB))
    [c,t] = C.shape
    slack = np.zeros((c,1));
    lambda1 = np.zeros((c,1));
    lambda2 = np.zeros((c,1));
    viol = -1*C[:,3]*C[:,2];
    viol = viol.T;

    for i in range(maxit):
        curri = np.argmax(viol);
        mx = viol[curri]
        if i%1000 == 0:
            print('Iteration {}, maxviol {}'.format(i, mx))
        
        if mx < thresh:
            break;
    
        p1 = C[curri,0];
        p2 = C[curri,1];
        b = C[curri,2];
        s = C[curri,3];
        kx = KA[p1-1,:];
        ky = KB[:,p2-1];

        alpha = min(lambda1[curri],(np.matmul(s,(b-np.matmul(kx, np.matmul(S,ky))-slack[curri]))) / \
                    (1/gamma + np.matmul(KA[p1,p1],KB[p2,p2])) );
        lambda1[curri] = lambda1[curri] - alpha;
        S[p1,p2] = S[p1,p2] + s*alpha;
        slack[curri] = slack[curri] - alpha/gamma;
        alpha2 =  min(lambda2[curri],gamma*slack[curri]);
        slack[curri] = slack[curri] - alpha2/gamma;
        lambda2[curri] = lambda2[curri] - alpha2;

#         update viols
#         v = KA[C[:,1],p1];
#         w = KB[p2,C[:,2].T;
        viol = viol + np.matmul(s,  alpha *C[:,3].T * KA[C[:,0],p1].T * KB[p2,C[:,1]]);
        viol[curri] = viol[curri] + (alpha+alpha2)/gamma;
    return [S, slack, i]


In [14]:
def asymmetricFrob_slack_kernel(KA, C, gamma = None, thresh = 10e-3):
    '''Returns the Frobenius based transform kernel.'''
    if thresh is None:
        thresh=10e-3;

    if gamma is None:
        gamma = 1e1;

    maxit = 1e4;
    
    [nA,nA] = KA.shape
    S = np.zeros((nA,nA))
    [c,t] = C.shape
    slack = np.zeros((c,1));
    lambda1 = np.zeros((c,1));
    lambda2 = np.zeros((c,1));
    v = (C[:, 0] *nA) + C[:, 1]
    v = np.array(v, dtype = np.int)
    mf1= (KA.flatten(1)[0, v] - C[:, 2])
    mf2= C[:,3].reshape(1, -1)
    viol = np.multiply(mf1,mf2)
    viol = viol.T;
    for i in range(int(maxit)):
        curri = np.argmax(viol)
        mx = np.max(viol)
        if mx < thresh*1000:
            break;
    
        p1 = int(C[curri,0])
        p2 = int(C[curri,1])
        b = int(C[curri,2])
        s = int(C[curri,3])
        kx = KA[p1,:];
        ky = KA[:,p2];
        
        arg1 = lambda1[curri]
        arg2 = np.matmul(kx, np.matmul(S,ky)) 
        arg3 = (1/gamma + KA[p1,p1]*KA[p2,p2])
        print(arg1, arg2, arg3)
        
        arg4 = s * (b - KA[p1, p2]- arg2 -slack[curri])
        alpha = min(arg1, arg4 / arg3)
        
        
        lambda1[curri] = lambda1[curri] - alpha;
        S[p1,p2] = S[p1,p2] + s*alpha[0,0];
        
        slack[curri] = slack[curri] - alpha/gamma;

        alpha2 =  min(lambda2[curri],gamma*slack[curri]);
        slack[curri] = slack[curri] - alpha2/gamma;
        lambda2[curri] = lambda2[curri] - alpha2;

        # update viols
        C = np.array(C, dtype = np.int)
        v = KA[C[:,0],p1]
        w = KA[p2,C[:,1].T]
        
        arg1 = np.multiply(KA[C[:,0],p1].T, KA[p2,C[:,1]])
        arg2 = alpha[0,0]*np.multiply(C[:,3].T, arg1)
        viol = viol + (s*arg2).T
        
        viol[curri] = viol[curri] + (alpha+alpha2)/gamma
        
    return [S, slack, i]

In [15]:
def learnAsymmTransformWithSVM(XA, yA, XB, yB, params):
    '''Returns the learned transformation matrix according to the weights and parameters defined.'''
    dA = XA.shape[1];
    dB = XB.shape[1]
    
    # Get constraints for data
    C = GetConstraints(yA,yB);

    if dA != dB:
        K0aa = formKernel(XA, XA, params);
        K0bb = formKernel(XB, XB, params);

        C[:,1] = C[:,1] - len(yA);
        S = AsymmetricFrob_slack_kernel2(K0aa,K0bb,C,params['gamma'],10e-3);
        params['S'] = S;

        L = np.eye(dA, dB) + np.matmul(XA.T * np.matmul(S, XB))

    else:
        X = np.vstack((XA, XB))

        # Get normalised data
        K0train = formKernel(X, X, params)

        # Function to perform frobenius based transformation learning of parameters
        S,_,_ = asymmetricFrob_slack_kernel(K0train,C,params['gamma'],10e-3);

        L = np.eye(dA) + np.matmul(X.T, np.matmul(S, X));
    return L

In [16]:
def AugmentWithOnes(data):
    return np.hstack((data, np.ones(((data.shape[0], 1)))))

def GetConstraints(y1, y2):
    '''
    Input :: 
        y1(list) : Labels available for the data
        y2(list) : Labels for training data
    Output :: 
        C(np.ndarray) : Constraint matrix built for the data
    '''
    pos=0;
    ly1=len(y1);
    ly2=len(y2);
    C=np.zeros((ly1*ly2,4));
    for i in range(ly1):
        for j in range(ly2):
            if(y1[i]==y2[j]):
                C[pos,:]=[i, j+ly1, 1, -1]; # w'Ax > 1; -w'Ax < -1;
            else:
                C[pos,:]=[i, j+ly1, -1, 1]; # w'Ax < -1
            pos=pos+1
    return C

In [17]:
def TrainMmdt(labels, data, param):
    '''
    Main function for training the mmdt model and transformation
    matrix.
    
    Input::
        labels(dictionary) : Contains labels for the data
        data(np.ndarray) : ndarray containing the data
        param(dictionary) : Dictionary containing parameters for processing
    Output::
        model : Returns trained model
        A : Transformation matrix'''    
    if ('C_s' not in param or 'C_t' not in param):
        param['C_s'] = 1
        param['C_t'] = 1
    
    if 'gamma' not in param:
        param['gamma'] = 1e-4
    
    dA = data['source'].shape[1]
    dB = data['target'].shape[1]
    
    param['A'] = np.eye(dB + 1, dA + 1)
    
    if 'train_classes' not in param:
        param['train_classes'] = np.sort(np.unique(labels['source']))
    
    for idx in range(param['mmdt_iter']):
        [model, data, param] = TrainMmdtOneIter(labels, data, param, idx)
    
    model.weights = model.weights * param['A'].T
    A = param['A']
    return [model, A]

In [18]:
def princomp(A):
    '''Returns the principal component of the data.'''
    M = (A - np.mean(A.T, axis=1).reshape(1, -1)).T
    [latent, coeff] = np.linalg.eig(np.cov(M))
    score = np.dot(coeff.T, M) # projection of the data in the new space
    args = np.argsort(-latent)
    coeff = coeff[:, args]
    return coeff, score, latent

In [19]:
def TrainMmdtOneIter(labels, data, param, idx):
    '''
    Runs one training iteration on the data which comprises of the model
    training and then learning the transformation matrix based on the 
    predicted model parameters
    
    Input::
        labels(dictionary) : Contains labels for the data
        data(np.ndarray) : ndarray containing the data
        param(dictionary) : Dictionary containing parameters for processing
        idx : iteration number
    Output::
        model : Returns trained model
        A : Transformation matrix'''    
    data['transformed_target'] =  AugmentWithOnes(data['target'])*param['A']
    data_svm = np.vstack((AugmentWithOnes(data['source']), data['transformed_target']))

    labels_svm = np.hstack((labels['source'].T, labels['target'].T))
    
    weights_s = param['C_s']* np.ones((labels['source'].size, 1))
    weights_t = param['C_t']* np.ones((labels['target'].size, 1))
    param['weights'] = np.vstack((weights_s, weights_t))
    
    # Train model based on the weights which depend on hyperparameters C_s and C_t
    # Iteration Step 1
    model = train(list(param['weights']), list(labels_svm.T), sps.csr_matrix(data_svm), '-c 1 -q')
    
    
    tstart = time.time()
    weights = []
    for i in range(0, 210):
        weights.append(model.w[i])
    
    # Learn transformation matrix based on the weights obtained by training in step 1
    # Iteration Step 2
    L = learnAsymmTransformWithSVM(np.array(weights).reshape(10, -1), \
    param['train_classes'], AugmentWithOnes(data['target']), labels['target'], param)
    param['A'] = L.T
    param['telapsed']['idx'] = time.time() - tstart;
    model.weights = np.array(weights).reshape(10, -1)
    return [model, data, param]

In [20]:
d1 = [0,1,2,3]
d2 = [0,1,2,3]
all_models = []
# Try for all possible cominations of source and target
for src in d1:
    for tar in d2:
        if (tar==src): continue
        print('='*55)
        print('Source ', src, 'Target', tar)
        global param
        param = Config(source=src, target=tar)
        source_domain = param['source']
        target_domain = param['target']
        
        # Load splits based on filename and update train and test ids
        splits = scipy.io.loadmat(param['result_filename'])
        train_ids = splits['train']
        test_ids = splits['test']
        train_ids_source = train_ids[0][0][0][0]
        train_ids_target = train_ids[0][0][1][0]
        test_ids_source = test_ids[0][0][0][0]
        test_ids_target = test_ids[0][0][1][0]
        
        # cur_models  : dictionary storing the models created based on grid search 
        # on the values of C_s and C_t for the source and target domains.
        cur_models={}
        cur_models['src'] = src
        cur_models['tar'] = tar
        elaps = 20
        C_s = [0.05, 0.1, 0.01, 0.5]
        C_t = [1, 5, 0.5, 2.5]
        models = []
        best_model = None
        m_acc = -np.inf
        for c_s in C_s:
            for c_t in C_t:
                param['C_s'], param['C_t'] = c_s, c_t
                for i in range(0, elaps):
                    # Data Loading
                    data = {}
                    data['train'] = {}
                    data['test'] = {}
                    print(i)
                    data['train']['source'] = Data[source_domain][train_ids_source[i]-1][0]
                    data['train']['target'] = Data[target_domain][train_ids_target[i]-1][0]
                    data['test']['target'] = Data[target_domain][test_ids_target[i]-1][0]
                    labels = {}
                    labels['train'] = {}
                    labels['test'] = {}
                    labels['train']['source'] = Labels[source_domain][train_ids_source[i]-1][0]
                    labels['train']['target'] = Labels[target_domain][train_ids_target[i]-1][0]
                    labels['test']['target'] = Labels[target_domain][test_ids_target[i]-1][0]
                    labels = UpdateLabelValues(labels, param)
                    
                    if param['dim'] < np.shape(data['train']['source'])[1]:
                        arr = np.array(data['train']['source'])
                        arr = np.vstack((arr, data['train']['target'])) 
                        arr = np.vstack((arr, data['test']['target']))
                        
                        # PCA to reduce diensionality of data
                        # We pick first 20 components
                        P, _, _ = princomp(arr)
                        data['train']['source'] = np.matmul(data['train']['source'], P[:, :20])
                        data['train']['target'] = np.matmul(data['train']['target'], P[:, :20])
                        data['test']['target'] = np.matmul(data['test']['target'], P[:, :20])
                    
                    # Main function call for getting the trained models
                    [model_mmdt, W] = TrainMmdt(labels['train'], data['train'], param)
                    arg1 = data['test']['target']
                    arg2 = np.ones((np.size(labels['test']['target']),1))
                    arg3 = np.concatenate((arg1, arg2), 1)

                    labels['test']['target'] = labels['test']['target'].reshape(-1,)
                    [pl, acc, pe] = predict(labels['test']['target'], arg3, model_mmdt);
                    accuracy[i] = acc[0];
                    print('Accuracy = {} (Time = {})\n'.format(accuracy[i], telapsed[i]));
                print('Mean Accuracy = {} +/- {}  (Mean time = {})'.format(np.mean(accuracy), np.std(accuracy)/np.sqrt(elaps), np.mean(telapsed)))
                mod = {}
                mod['C_s'], mod['C_t'], mod['accuracy'] = c_s, c_t, np.mean(accuracy)
                models.append(mod)
                # Get the best model based on mean accuracy over elaps
                if(mod['accuracy'] > m_acc):
                    m_acc = mod['accuracy']
                    best_model = mod
        print(best_model)
        cur_models['models'] = models
        cur_models['best_model'] = best_model
        all_models.append(cur_models)

Source  0 Target 1
0
Accuracy = 61.1321% (162/265) (classification)
Accuracy = [61.13207547] (Time = [0.])

1
Accuracy = 64.5283% (171/265) (classification)
Accuracy = [64.52830189] (Time = [0.])

2
Accuracy = 67.5472% (179/265) (classification)
Accuracy = [67.54716981] (Time = [0.])

3
Accuracy = 64.1509% (170/265) (classification)
Accuracy = [64.1509434] (Time = [0.])

4
Accuracy = 72.4528% (192/265) (classification)
Accuracy = [72.45283019] (Time = [0.])

5
Accuracy = 75.4717% (200/265) (classification)
Accuracy = [75.47169811] (Time = [0.])

6
Accuracy = 67.1698% (178/265) (classification)
Accuracy = [67.16981132] (Time = [0.])

7
Accuracy = 66.4151% (176/265) (classification)
Accuracy = [66.41509434] (Time = [0.])

8
Accuracy = 68.6792% (182/265) (classification)
Accuracy = [68.67924528] (Time = [0.])

9
Accuracy = 60% (159/265) (classification)
Accuracy = [60.] (Time = [0.])

10
Accuracy = 68.6792% (182/265) (classification)
Accuracy = [68.67924528] (Time = [0.])

11
Accuracy = 6

Accuracy = 64.9057% (172/265) (classification)
Accuracy = [64.90566038] (Time = [0.])

9
Accuracy = 58.8679% (156/265) (classification)
Accuracy = [58.86792453] (Time = [0.])

10
Accuracy = 66.7925% (177/265) (classification)
Accuracy = [66.79245283] (Time = [0.])

11
Accuracy = 60.7547% (161/265) (classification)
Accuracy = [60.75471698] (Time = [0.])

12
Accuracy = 75.0943% (199/265) (classification)
Accuracy = [75.09433962] (Time = [0.])

13
Accuracy = 54.3396% (144/265) (classification)
Accuracy = [54.33962264] (Time = [0.])

14
Accuracy = 67.9245% (180/265) (classification)
Accuracy = [67.9245283] (Time = [0.])

15
Accuracy = 59.6226% (158/265) (classification)
Accuracy = [59.62264151] (Time = [0.])

16
Accuracy = 60% (159/265) (classification)
Accuracy = [60.] (Time = [0.])

17
Accuracy = 69.8113% (185/265) (classification)
Accuracy = [69.81132075] (Time = [0.])

18
Accuracy = 63.0189% (167/265) (classification)
Accuracy = [63.01886792] (Time = [0.])

19
Accuracy = 58.1132% (154/

Accuracy = 68.6792% (182/265) (classification)
Accuracy = [68.67924528] (Time = [0.])

18
Accuracy = 66.4151% (176/265) (classification)
Accuracy = [66.41509434] (Time = [0.])

19
Accuracy = 63.0189% (167/265) (classification)
Accuracy = [63.01886792] (Time = [0.])

Mean Accuracy = 65.03773584905659 +/- 0.9313475883721813  (Mean time = 0.0)
0
Accuracy = 60.7547% (161/265) (classification)
Accuracy = [60.75471698] (Time = [0.])

1
Accuracy = 61.5094% (163/265) (classification)
Accuracy = [61.50943396] (Time = [0.])

2
Accuracy = 61.8868% (164/265) (classification)
Accuracy = [61.88679245] (Time = [0.])

3
Accuracy = 62.2642% (165/265) (classification)
Accuracy = [62.26415094] (Time = [0.])

4
Accuracy = 71.6981% (190/265) (classification)
Accuracy = [71.69811321] (Time = [0.])

5
Accuracy = 69.8113% (185/265) (classification)
Accuracy = [69.81132075] (Time = [0.])

6
Accuracy = 64.9057% (172/265) (classification)
Accuracy = [64.90566038] (Time = [0.])

7
Accuracy = 66.7925% (177/265) (c

Accuracy = 69.0566% (183/265) (classification)
Accuracy = [69.05660377] (Time = [0.])

6
Accuracy = 66.0377% (175/265) (classification)
Accuracy = [66.03773585] (Time = [0.])

7
Accuracy = 62.6415% (166/265) (classification)
Accuracy = [62.64150943] (Time = [0.])

8
Accuracy = 61.5094% (163/265) (classification)
Accuracy = [61.50943396] (Time = [0.])

9
Accuracy = 54.3396% (144/265) (classification)
Accuracy = [54.33962264] (Time = [0.])

10
Accuracy = 63.3962% (168/265) (classification)
Accuracy = [63.39622642] (Time = [0.])

11
Accuracy = 60% (159/265) (classification)
Accuracy = [60.] (Time = [0.])

12
Accuracy = 70.9434% (188/265) (classification)
Accuracy = [70.94339623] (Time = [0.])

13
Accuracy = 53.9623% (143/265) (classification)
Accuracy = [53.96226415] (Time = [0.])

14
Accuracy = 63.7736% (169/265) (classification)
Accuracy = [63.77358491] (Time = [0.])

15
Accuracy = 54.717% (145/265) (classification)
Accuracy = [54.71698113] (Time = [0.])

16
Accuracy = 55.8491% (148/265

Accuracy = 55.1181% (70/127) (classification)
Accuracy = [55.11811024] (Time = [0.])

14
Accuracy = 44.8819% (57/127) (classification)
Accuracy = [44.88188976] (Time = [0.])

15
Accuracy = 56.6929% (72/127) (classification)
Accuracy = [56.69291339] (Time = [0.])

16
Accuracy = 57.4803% (73/127) (classification)
Accuracy = [57.48031496] (Time = [0.])

17
Accuracy = 56.6929% (72/127) (classification)
Accuracy = [56.69291339] (Time = [0.])

18
Accuracy = 58.2677% (74/127) (classification)
Accuracy = [58.26771654] (Time = [0.])

19
Accuracy = 52.7559% (67/127) (classification)
Accuracy = [52.75590551] (Time = [0.])

Mean Accuracy = 53.85826771653543 +/- 1.0555293592716675  (Mean time = 0.0)
0
Accuracy = 57.4803% (73/127) (classification)
Accuracy = [57.48031496] (Time = [0.])

1
Accuracy = 56.6929% (72/127) (classification)
Accuracy = [56.69291339] (Time = [0.])

2
Accuracy = 62.9921% (80/127) (classification)
Accuracy = [62.99212598] (Time = [0.])

3
Accuracy = 62.2047% (79/127) (classifi

...

Accuracy = 63.3962% (168/265) (classification)
Accuracy = [63.39622642] (Time = [0.])

1
Accuracy = 61.5094% (163/265) (classification)
Accuracy = [61.50943396] (Time = [0.])

2
Accuracy = 54.717% (145/265) (classification)
Accuracy = [54.71698113] (Time = [0.])

3
Accuracy = 60.3774% (160/265) (classification)
Accuracy = [60.37735849] (Time = [0.])

4
Accuracy = 58.4906% (155/265) (classification)
Accuracy = [58.49056604] (Time = [0.])

5
Accuracy = 58.1132% (154/265) (classification)
Accuracy = [58.11320755] (Time = [0.])

6
Accuracy = 52.4528% (139/265) (classification)
Accuracy = [52.45283019] (Time = [0.])

7
Accuracy = 45.283% (120/265) (classification)
Accuracy = [45.28301887] (Time = [0.])

8
Accuracy = 53.9623% (143/265) (classification)
Accuracy = [53.96226415] (Time = [0.])

9
Accuracy = 58.4906% (155/265) (classification)
Accuracy = [58.49056604] (Time = [0.])

10
Accuracy = 56.2264% (149/265) (classification)
Accuracy = [56.22641509] (Time = [0.])

11
Accuracy = 63.7736% (

Accuracy = 47.2441% (60/127) (classification)
Accuracy = [47.24409449] (Time = [0.])

9
Accuracy = 55.1181% (70/127) (classification)
Accuracy = [55.11811024] (Time = [0.])

10
Accuracy = 55.1181% (70/127) (classification)
Accuracy = [55.11811024] (Time = [0.])

11
Accuracy = 55.1181% (70/127) (classification)
Accuracy = [55.11811024] (Time = [0.])

12
Accuracy = 58.2677% (74/127) (classification)
Accuracy = [58.26771654] (Time = [0.])

13
Accuracy = 55.1181% (70/127) (classification)
Accuracy = [55.11811024] (Time = [0.])

14
Accuracy = 47.2441% (60/127) (classification)
Accuracy = [47.24409449] (Time = [0.])

15
Accuracy = 52.7559% (67/127) (classification)
Accuracy = [52.75590551] (Time = [0.])

16
Accuracy = 59.8425% (76/127) (classification)
Accuracy = [59.84251969] (Time = [0.])

17
Accuracy = 50.3937% (64/127) (classification)
Accuracy = [50.39370079] (Time = [0.])

18
Accuracy = 49.6063% (63/127) (classification)
Accuracy = [49.60629921] (Time = [0.])

19
Accuracy = 47.2441% (6

Accuracy = 51.9685% (66/127) (classification)
Accuracy = [51.96850394] (Time = [0.])

19
Accuracy = 48.8189% (62/127) (classification)
Accuracy = [48.81889764] (Time = [0.])

Mean Accuracy = 54.88188976377953 +/- 0.8464109125140495  (Mean time = 0.0)
0
Accuracy = 52.7559% (67/127) (classification)
Accuracy = [52.75590551] (Time = [0.])

1
Accuracy = 52.7559% (67/127) (classification)
Accuracy = [52.75590551] (Time = [0.])

2
Accuracy = 55.9055% (71/127) (classification)
Accuracy = [55.90551181] (Time = [0.])

3
Accuracy = 50.3937% (64/127) (classification)
Accuracy = [50.39370079] (Time = [0.])

4
Accuracy = 58.2677% (74/127) (classification)
Accuracy = [58.26771654] (Time = [0.])

5
Accuracy = 48.8189% (62/127) (classification)
Accuracy = [48.81889764] (Time = [0.])

6
Accuracy = 59.0551% (75/127) (classification)
Accuracy = [59.05511811] (Time = [0.])

7
Accuracy = 58.2677% (74/127) (classification)
Accuracy = [58.26771654] (Time = [0.])

8
Accuracy = 45.6693% (58/127) (classificatio

Accuracy = 53.5433% (68/127) (classification)
Accuracy = [53.54330709] (Time = [0.])

8
Accuracy = 41.7323% (53/127) (classification)
Accuracy = [41.73228346] (Time = [0.])

9
Accuracy = 55.1181% (70/127) (classification)
Accuracy = [55.11811024] (Time = [0.])

10
Accuracy = 48.8189% (62/127) (classification)
Accuracy = [48.81889764] (Time = [0.])

11
Accuracy = 50.3937% (64/127) (classification)
Accuracy = [50.39370079] (Time = [0.])

12
Accuracy = 44.0945% (56/127) (classification)
Accuracy = [44.09448819] (Time = [0.])

13
Accuracy = 50.3937% (64/127) (classification)
Accuracy = [50.39370079] (Time = [0.])

14
Accuracy = 48.0315% (61/127) (classification)
Accuracy = [48.03149606] (Time = [0.])

15
Accuracy = 44.8819% (57/127) (classification)
Accuracy = [44.88188976] (Time = [0.])

16
Accuracy = 48.8189% (62/127) (classification)
Accuracy = [48.81889764] (Time = [0.])

17
Accuracy = 48.0315% (61/127) (classification)
Accuracy = [48.03149606] (Time = [0.])

18
Accuracy = 40.1575% (51

Accuracy = 50.3937% (64/127) (classification)
Accuracy = [50.39370079] (Time = [0.])

18
Accuracy = 45.6693% (58/127) (classification)
Accuracy = [45.66929134] (Time = [0.])

19
Accuracy = 48.0315% (61/127) (classification)
Accuracy = [48.03149606] (Time = [0.])

Mean Accuracy = 51.220472440944874 +/- 0.8580058113718957  (Mean time = 0.0)
0
Accuracy = 53.5433% (68/127) (classification)
Accuracy = [53.54330709] (Time = [0.])

1
Accuracy = 48.0315% (61/127) (classification)
Accuracy = [48.03149606] (Time = [0.])

2
Accuracy = 54.3307% (69/127) (classification)
Accuracy = [54.33070866] (Time = [0.])

3
Accuracy = 49.6063% (63/127) (classification)
Accuracy = [49.60629921] (Time = [0.])

4
Accuracy = 44.8819% (57/127) (classification)
Accuracy = [44.88188976] (Time = [0.])

5
Accuracy = 48.0315% (61/127) (classification)
Accuracy = [48.03149606] (Time = [0.])

6
Accuracy = 48.0315% (61/127) (classification)
Accuracy = [48.03149606] (Time = [0.])

7
Accuracy = 55.1181% (70/127) (classificat

In [21]:
print("|C_s\t|C_t\t|Accuracy\t|")
print(('|' + ('-' * 7))*3 + ('-' * 8) + '|')
for mod in models:
    print('|' + str(mod['C_s']) + '\t|' + str(mod['C_t']) + '\t|' +  "%.3f" % mod['accuracy'] + "\t\t" + '|' )

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|54.252		|
|0.05	|5	|51.260		|
|0.05	|0.5	|55.984		|
|0.05	|2.5	|52.520		|
|0.1	|1	|54.882		|
|0.1	|5	|51.811		|
|0.1	|0.5	|55.315		|
|0.1	|2.5	|52.953		|
|0.01	|1	|50.591		|
|0.01	|5	|48.346		|
|0.01	|0.5	|52.598		|
|0.01	|2.5	|48.898		|
|0.5	|1	|50.787		|
|0.5	|5	|51.220		|
|0.5	|0.5	|49.843		|
|0.5	|2.5	|51.654		|


|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|66.811		|
|0.05	|5	|63.622		|
|0.05	|0.5	|68.661		|
|0.05	|2.5	|64.803		|
|0.1	|1	|67.874		|
|0.1	|5	|65.591		|
|0.1	|0.5	|70.118		|
|0.1	|2.5	|66.260		|
|0.01	|1	|59.843		|
|0.01	|5	|57.756		|
|0.01	|0.5	|61.614		|
|0.01	|2.5	|58.543		|
|0.5	|1	|68.898		|
|0.5	|5	|66.181		|
|0.5	|0.5	|69.606		|
|0.5	|2.5	|66.535		|

In [40]:
dicc = {0: 'amazon', 1: 'webcam', 2: 'dslr', 3:'caltech'}
for cur_model in all_models:
    src = dicc[cur_model['src']]
    tar = dicc[cur_model['tar']]
    print('#### Source : ', src, 'Target : ', tar)
    print()
    print("|C_s\t|C_t\t|Accuracy\t|")
    print(('|' + ('-' * 7))*3 + ('-' * 8) + '|')
    for mod in cur_model['models']:
        print('|' + str(mod['C_s']) + '\t|' + str(mod['C_t']) + '\t|' + "%.3f" % mod['accuracy'] + "\t\t" + '|' )
    
    print()
    best = cur_model['best_model']
    print("##### best_model =>")
    print("**C_s :** {} ".format(best['C_s']) + ("&nbsp;" * 11) +  "**C_t :** {0} ".format(best['C_t']) + ("&nbsp;" * 11) + "**Accuracy :** {0:.3f}".format(best['accuracy']))
    print()

#### Source :  amazon Target :  webcam

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|65.717		|
|0.05	|5	|64.377		|
|0.05	|0.5	|65.717		|
|0.05	|2.5	|64.792		|
|0.1	|1	|64.453		|
|0.1	|5	|64.264		|
|0.1	|0.5	|63.453		|
|0.1	|2.5	|64.377		|
|0.01	|1	|65.038		|
|0.01	|5	|63.906		|
|0.01	|0.5	|65.943		|
|0.01	|2.5	|64.377		|
|0.5	|1	|56.000		|
|0.5	|5	|61.283		|
|0.5	|0.5	|51.434		|
|0.5	|2.5	|60.642		|

##### best_model =>
**C_s :** 0.01 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**C_t :** 0.5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Accuracy :** 65.943

#### Source :  amazon Target :  dslr

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|55.906		|
|0.05	|5	|53.858		|
|0.05	|0.5	|55.551		|
|0.05	|2.5	|54.409		|
|0.1	|1	|55.157		|
|0.1	|5	|54.449		|
|0.1	|0.5	|54.488		|
|0.1	|2.5	|55.118		|
|0.01	|1	|53.937		|
|0.01	|5	|51.890		|
|0.01	|0.5	|54.961		|
|0.01	|2.5	|52.520		|
|0.5	|1	|50.197		|
|0.5	|5	|5

#### Source :  amazon Target :  webcam

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|65.717		|
|0.05	|5	|64.377		|
|0.05	|0.5	|65.717		|
|0.05	|2.5	|64.792		|
|0.1	|1	|64.453		|
|0.1	|5	|64.264		|
|0.1	|0.5	|63.453		|
|0.1	|2.5	|64.377		|
|0.01	|1	|65.038		|
|0.01	|5	|63.906		|
|0.01	|0.5	|65.943		|
|0.01	|2.5	|64.377		|
|0.5	|1	|56.000		|
|0.5	|5	|61.283		|
|0.5	|0.5	|51.434		|
|0.5	|2.5	|60.642		|

##### best_model =>
**C_s :** 0.01 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**C_t :** 0.5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Accuracy :** 65.943

#### Source :  amazon Target :  dslr

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|55.906		|
|0.05	|5	|53.858		|
|0.05	|0.5	|55.551		|
|0.05	|2.5	|54.409		|
|0.1	|1	|55.157		|
|0.1	|5	|54.449		|
|0.1	|0.5	|54.488		|
|0.1	|2.5	|55.118		|
|0.01	|1	|53.937		|
|0.01	|5	|51.890		|
|0.01	|0.5	|54.961		|
|0.01	|2.5	|52.520		|
|0.5	|1	|50.197		|
|0.5	|5	|52.874		|
|0.5	|0.5	|47.677		|
|0.5	|2.5	|52.244		|

##### best_model =>
**C_s :** 0.05 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**C_t :** 1 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Accuracy :** 55.906

#### Source :  amazon Target :  caltech

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|40.503		|
|0.05	|5	|37.781		|
|0.05	|0.5	|42.145		|
|0.05	|2.5	|38.591		|
|0.1	|1	|41.862		|
|0.1	|5	|38.843		|
|0.1	|0.5	|43.184		|
|0.1	|2.5	|40.091		|
|0.01	|1	|36.171		|
|0.01	|5	|34.318		|
|0.01	|0.5	|37.649		|
|0.01	|2.5	|34.895		|
|0.5	|1	|42.447		|
|0.5	|5	|41.080		|
|0.5	|0.5	|42.425		|
|0.5	|2.5	|41.743		|

##### best_model =>
**C_s :** 0.1 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**C_t :** 0.5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Accuracy :** 43.184

#### Source :  webcam Target :  amazon

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|50.216		|
|0.05	|5	|49.256		|
|0.05	|0.5	|51.013		|
|0.05	|2.5	|49.526		|
|0.1	|1	|49.353		|
|0.1	|5	|48.362		|
|0.1	|0.5	|49.731		|
|0.1	|2.5	|48.648		|
|0.01	|1	|49.246		|
|0.01	|5	|48.276		|
|0.01	|0.5	|50.253		|
|0.01	|2.5	|48.491		|
|0.5	|1	|46.202		|
|0.5	|5	|45.954		|
|0.5	|0.5	|45.603		|
|0.5	|2.5	|46.083		|

##### best_model =>
**C_s :** 0.05 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**C_t :** 0.5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Accuracy :** 51.013

#### Source :  webcam Target :  dslr

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|66.811		|
|0.05	|5	|63.504		|
|0.05	|0.5	|68.701		|
|0.05	|2.5	|64.370		|
|0.1	|1	|67.717		|
|0.1	|5	|65.512		|
|0.1	|0.5	|70.157		|
|0.1	|2.5	|66.417		|
|0.01	|1	|59.685		|
|0.01	|5	|57.598		|
|0.01	|0.5	|61.654		|
|0.01	|2.5	|58.583		|
|0.5	|1	|68.740		|
|0.5	|5	|66.339		|
|0.5	|0.5	|69.488		|
|0.5	|2.5	|66.772		|

##### best_model =>
**C_s :** 0.1 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**C_t :** 0.5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Accuracy :** 70.157

#### Source :  webcam Target :  caltech

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|34.245		|
|0.05	|5	|32.644		|
|0.05	|0.5	|35.554		|
|0.05	|2.5	|33.088		|
|0.1	|1	|35.256		|
|0.1	|5	|33.522		|
|0.1	|0.5	|36.176		|
|0.1	|2.5	|34.053		|
|0.01	|1	|30.645		|
|0.01	|5	|29.602		|
|0.01	|0.5	|31.766		|
|0.01	|2.5	|29.817		|
|0.5	|1	|34.849		|
|0.5	|5	|33.683		|
|0.5	|0.5	|35.256		|
|0.5	|2.5	|34.209		|

##### best_model =>
**C_s :** 0.1 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**C_t :** 0.5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Accuracy :** 36.176

#### Source :  dslr Target :  amazon

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|50.323		|
|0.05	|5	|49.321		|
|0.05	|0.5	|51.072		|
|0.05	|2.5	|49.520		|
|0.1	|1	|50.469		|
|0.1	|5	|49.558		|
|0.1	|0.5	|51.061		|
|0.1	|2.5	|49.747		|
|0.01	|1	|47.780		|
|0.01	|5	|47.047		|
|0.01	|0.5	|48.658		|
|0.01	|2.5	|47.214		|
|0.5	|1	|48.561		|
|0.5	|5	|47.953		|
|0.5	|0.5	|47.877		|
|0.5	|2.5	|48.211		|

##### best_model =>
**C_s :** 0.05 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**C_t :** 0.5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Accuracy :** 51.072

#### Source :  dslr Target :  webcam

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|75.887		|
|0.05	|5	|73.377		|
|0.05	|0.5	|77.830		|
|0.05	|2.5	|74.094		|
|0.1	|1	|77.887		|
|0.1	|5	|75.547		|
|0.1	|0.5	|79.151		|
|0.1	|2.5	|76.321		|
|0.01	|1	|68.453		|
|0.01	|5	|66.887		|
|0.01	|0.5	|70.094		|
|0.01	|2.5	|67.358		|
|0.5	|1	|78.038		|
|0.5	|5	|76.755		|
|0.5	|0.5	|78.642		|
|0.5	|2.5	|77.226		|

##### best_model =>
**C_s :** 0.1 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**C_t :** 0.5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Accuracy :** 79.151

#### Source :  dslr Target :  caltech

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|35.091		|
|0.05	|5	|33.527		|
|0.05	|0.5	|36.555		|
|0.05	|2.5	|33.980		|
|0.1	|1	|36.258		|
|0.1	|5	|34.378		|
|0.1	|0.5	|37.191		|
|0.1	|2.5	|34.913		|
|0.01	|1	|31.825		|
|0.01	|5	|30.773		|
|0.01	|0.5	|32.832		|
|0.01	|2.5	|31.057		|
|0.5	|1	|35.636		|
|0.5	|5	|34.657		|
|0.5	|0.5	|35.851		|
|0.5	|2.5	|35.114		|

##### best_model =>
**C_s :** 0.1 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**C_t :** 0.5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Accuracy :** 37.191

#### Source :  caltech Target :  amazon

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|51.859		|
|0.05	|5	|50.361		|
|0.05	|0.5	|52.565		|
|0.05	|2.5	|50.830		|
|0.1	|1	|52.069		|
|0.1	|5	|50.927		|
|0.1	|0.5	|52.484		|
|0.1	|2.5	|51.298		|
|0.01	|1	|49.025		|
|0.01	|5	|47.877		|
|0.01	|0.5	|49.952		|
|0.01	|2.5	|48.136		|
|0.5	|1	|49.617		|
|0.5	|5	|49.327		|
|0.5	|0.5	|48.459		|
|0.5	|2.5	|49.607		|

##### best_model =>
**C_s :** 0.05 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**C_t :** 0.5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Accuracy :** 52.565

#### Source :  caltech Target :  webcam

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|64.321		|
|0.05	|5	|62.226		|
|0.05	|0.5	|65.132		|
|0.05	|2.5	|63.019		|
|0.1	|1	|63.358		|
|0.1	|5	|62.094		|
|0.1	|0.5	|64.226		|
|0.1	|2.5	|62.679		|
|0.01	|1	|61.151		|
|0.01	|5	|59.792		|
|0.01	|0.5	|62.547		|
|0.01	|2.5	|60.189		|
|0.5	|1	|56.509		|
|0.5	|5	|59.132		|
|0.5	|0.5	|53.264		|
|0.5	|2.5	|58.679		|

##### best_model =>
**C_s :** 0.05 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**C_t :** 0.5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Accuracy :** 65.132

#### Source :  caltech Target :  dslr

|C_s	|C_t	|Accuracy	|
|-------|-------|---------------|
|0.05	|1	|54.252		|
|0.05	|5	|51.260		|
|0.05	|0.5	|55.984		|
|0.05	|2.5	|52.520		|
|0.1	|1	|54.882		|
|0.1	|5	|51.811		|
|0.1	|0.5	|55.315		|
|0.1	|2.5	|52.953		|
|0.01	|1	|50.591		|
|0.01	|5	|48.346		|
|0.01	|0.5	|52.598		|
|0.01	|2.5	|48.898		|
|0.5	|1	|50.787		|
|0.5	|5	|51.220		|
|0.5	|0.5	|49.843		|
|0.5	|2.5	|51.654		|

##### best_model =>
**C_s :** 0.05 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**C_t :** 0.5 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Accuracy :** 55.984